In [1]:
from networkx import DiGraph
from networkx.algorithms.simple_paths import all_simple_paths
from max_lambda2 import Graph
import xml.etree.ElementTree as ET
from tqdm.notebook import trange, tqdm
import sumolib
import json

tree = ET.parse('../scenario/lust.net.xml')
root = tree.getroot()
network_rep = Graph()
for edge in root.findall("edge"):
    if 'from' in edge.attrib:
        network_rep.addEdge(edge.attrib['from'], edge.attrib['to'],edge.attrib['id'])

min_subnet = {}
max_lmbd = 0
net = sumolib.net.readNet('../scenario/lust.net.xml')
vulnerable_edges = [e for e in net.getEdges() if e.getLaneNumber()>2]

for disrupted_edge in tqdm(vulnerable_edges):
    disrupted_edge = disrupted_edge.getID()
    for lmbd in range(1,50):
        subnetwork_edges = network_rep.getSubnet(disrupted_edge, lmbd)

        G = DiGraph()
        for edge in subnetwork_edges:
            if disrupted_edge != edge:
                G.add_edge(*network_rep.nodes[edge])
        source, dest = network_rep.nodes[disrupted_edge]
        if len(list(all_simple_paths(G, source, dest))) > 0:
            if lmbd > 1:
                min_subnet[disrupted_edge] = lmbd 
            else:
                min_subnet[disrupted_edge] = lmbd 
            if lmbd > max_lmbd:
                max_lmbd = lmbd
            break
json.dump(min_subnet, open('min_subnet.json'))

  0%|          | 0/823 [00:00<?, ?it/s]

KeyboardInterrupt: 